In [21]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [22]:
username = 'APTX9998'
password = 'Acp123456.'

In [23]:
max_tweets = 50000

In [24]:
output_path = r"D:\5.ACP"
output_filename = "CT.csv"
real_time_saving = "real_time_CT_data.csv"
output = f"{output_path}\\{output_filename}"
realtime_output = f"{output_path}\\{real_time_saving}"

In [25]:
options = webdriver.FirefoxOptions()
options.add_argument('--headless')

In [26]:
def scroll_down(browser):
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)

In [27]:
def write_data_entry(entry):
    with open(realtime_output, 'a') as f:
        f.write(entry+'\n')

In [28]:
def start_twitter_search():
    with webdriver.Firefox(options=options) as browser:
        url = 'https://twitter.com/Citi'
        browser.get(url)

        wait = WebDriverWait(browser, 10)

        login_button = wait.until(EC.presence_of_element_located((By.XPATH, '//a[@href="/login"]')))
        login_button.click()

        username_input = wait.until(EC.presence_of_element_located((By.XPATH, './/input[@name="text"]')))
        username_input.send_keys(username)
        username_input.send_keys(Keys.RETURN)

        time.sleep(3)

        password_input = wait.until(EC.presence_of_element_located((By.XPATH, './/input[@name="password"]')))
        password_input.send_keys(password)
        password_input.send_keys(Keys.RETURN)

        wait.until(EC.presence_of_element_located((By.XPATH, '//input[@enterkeyhint="search"]')))

        # Removing the topic search
        search_input = browser.find_element(By.XPATH, '//input[@enterkeyhint="search"]')
        search_input.send_keys(Keys.RETURN)

        current_tweets = 0
        user_data = []
        text_data = []
        time_data = []

        while current_tweets < max_tweets:
            for _ in range(10):
                scroll_down(browser)
    
            tweets = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//article[@role="article"]')))
    
            for tweet in tweets:
                try:
                    user = tweet.find_element(By.XPATH, './/span[contains(text(), "@")]').text
                    text = tweet.find_element(By.XPATH, ".//div[@lang]").text
                    tweet_time = tweet.find_element(By.XPATH, ".//time").get_attribute("datetime")
    
                    tweets_data = [user, text, tweet_time]
                except Exception as e:
                    print(f"Error extracting tweet: {e}")
                    tweets_data = ['user', 'text', "time"]
    
                userdata = tweets_data[0]
                textdata = " ".join(tweets_data[1].split())
                timedata = tweets_data[2]
                entry = userdata + ',' + textdata + ',' + timedata
                write_data_entry(entry)
                user_data.append(userdata)
                text_data.append(textdata)
                time_data.append(entry)
    
                current_tweets += 1

            print(f"Scraped {current_tweets} tweets")

        df = pd.DataFrame({'user': user_data, 'text': text_data, 'time': time_data})
        df.to_csv(output, index=False)
        print(f"Total {current_tweets} tweets scraped")

In [29]:
def main():
    start_twitter_search()

In [30]:
if __name__ == "__main__":
   main()

TimeoutException: Message: 
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:189:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:507:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:132:16


In [ ]:
df = pd.read_csv(r"D:\5.ACP\BOA.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100003 entries, 0 to 100002
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   user    100003 non-null  object
 1   text    100003 non-null  object
 2   time    100003 non-null  object
dtypes: object(3)
memory usage: 2.3+ MB


In [ ]:
dups = df.duplicated()

In [ ]:
# count the number of duplicated rows
dup_count = dups.sum()

In [ ]:
print(dup_count)

99984
